In [669]:
import pandas as pd
import numpy as np
import plotly.express as px
import klib as kl

# impute missing values with iterative imputer 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [670]:
products_data = pd.read_csv('data/regression_clean_data_final_1.csv')



In [671]:
data = pd.read_csv('data/site_changes_cleaned.csv')



# create a date column from month and year and add day as 1
data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))
data = kl.clean_column_names(data)

data.head()

,month,year,incentive,apr,notes,bank,notes_1,tag,site_changes,date
0,1,2021,0,0.0,NaN,abanca,NaN,NaN,NaN,2021-01-01
1,6,2021,150,0.0,800-1200 deposit,abanca,NaN,increase incentive,NaN,2021-06-01
2,8,2021,150,0.0,800-1200 deposit,abanca,NaN,incentive decrease,NaN,2021-08-01
3,4,2022,300,0.0,if over 1200 deposit,abanca,NaN,"incentive increase, apr decrease",NaN,2022-04-01
4,12,2022,300,0.0,mejor cambia de el sitio,abanca,NaN,NaN,major site change,2022-12-01


In [672]:
data.groupby('bank')['incentive'].sum()

bank
abanca           1500
banc sabadell    1050
bankinter           0
bbva                0
evobanco            0
myinvestor          0
n26                 0
openbank            0
revolut             0
santander         850
Name: incentive, dtype: int64

In [673]:
data.groupby('bank')['apr'].sum()

bank
abanca            4.00
banc sabadell    14.00
bankinter        15.00
bbva              6.00
evobanco          7.47
myinvestor        7.00
n26               2.26
openbank          5.14
revolut           0.00
santander         0.00
Name: apr, dtype: float64

In [674]:
data.tag.value_counts()

tag
increase incentive                  15
increase apr                        15
incentive decrease                   3
decrease apr                         3
incentive decrease, apr increase     2
incentive increase, apr decrease     1
incentive increase, apr increase     1
incentive decrease, apr decrease     1
temporary incentive                  1
Name: count, dtype: int64

In [675]:
data.head()

,month,year,incentive,apr,notes,bank,notes_1,tag,site_changes,date
0,1,2021,0,0.0,NaN,abanca,NaN,NaN,NaN,2021-01-01
1,6,2021,150,0.0,800-1200 deposit,abanca,NaN,increase incentive,NaN,2021-06-01
2,8,2021,150,0.0,800-1200 deposit,abanca,NaN,incentive decrease,NaN,2021-08-01
3,4,2022,300,0.0,if over 1200 deposit,abanca,NaN,"incentive increase, apr decrease",NaN,2022-04-01
4,12,2022,300,0.0,mejor cambia de el sitio,abanca,NaN,NaN,major site change,2022-12-01


In [676]:
# rename tags

data.tag = data.tag.replace({'apr increase':'increase apr', 'incentive increase':'increase incentive', 'decrease incentive':'incentive decrease',  'decrease incentive ':'incentive decrease',
'incentive decrease, apr increase':'decrease incentive, increase apr',
'incentive increase, apr decrease':'increase incentive, decrease apr',
'incentive increase, apr increase':'increase incentive, increase apr',
'incentive decrease, apr decrease':'decrease incentive, decrease apr'})

       
data.tag.value_counts()

tag
increase incentive                  15
increase apr                        15
incentive decrease                   3
decrease apr                         3
decrease incentive, increase apr     2
increase incentive, decrease apr     1
increase incentive, increase apr     1
decrease incentive, decrease apr     1
temporary incentive                  1
Name: count, dtype: int64

In [677]:
data.head()

,month,year,incentive,apr,notes,bank,notes_1,tag,site_changes,date
0,1,2021,0,0.0,NaN,abanca,NaN,NaN,NaN,2021-01-01
1,6,2021,150,0.0,800-1200 deposit,abanca,NaN,increase incentive,NaN,2021-06-01
2,8,2021,150,0.0,800-1200 deposit,abanca,NaN,incentive decrease,NaN,2021-08-01
3,4,2022,300,0.0,if over 1200 deposit,abanca,NaN,"increase incentive, decrease apr",NaN,2022-04-01
4,12,2022,300,0.0,mejor cambia de el sitio,abanca,NaN,NaN,major site change,2022-12-01


In [678]:
mapping_dict = {
    'abanca': 'abanca',
    'banc sabadell': 'banc sabadell',
    'bancosabadell': 'banc sabadell',
    'bancosantander': 'santander',
    'bancsabadell': 'banc sabadell',
    'sabadell': 'banc sabadell',
    'bank': 'bank',
    'bankinter': 'bankinter',
    'bbva': 'bbva',
    'caixabank': 'caixabank',
    'caixa_bank': 'caixabank',
    'date': 'date',
    'evobanco': 'evobanco',
    'evo_banco': 'evobanco',
    'imagin': 'imagin',
    'ing': 'ing',
    'ing-bank': 'ing',
    'ing_bank': 'ing',
    'kutxabank': 'kutxabank',
    'liberbank': 'liberbank',
    'myinvestor': 'myinvestor',
    'n26': 'n26',
    'n26.com': 'n26',
    None: None,  # handling nan (not a number)
    'openbank': 'openbank',
    'orangebank': 'orangebank',
    'pibank': 'pibank',
    'renaultbank': 'renaultbank',
    'revolut': 'revolut',
    'santander': 'santander',
    'value': 'value'
}

# map santander to bancosantander in data 
data.bank = data.bank.map(mapping_dict)
products_data.bank = products_data.bank.map(mapping_dict)



create two lists one for APR and Incentive increases. 

In [679]:
products_data.head()

,bank,date,web_traffic,cross_visitation,search_interest,rank,incentive,apr,mentions,app_installs,ppc_spend
0,abanca,2021-12-01,15202.983835,0.077994,7.75,4.0,150.0,0.0,1.0,43605.699637,1506.110026
1,abanca,2022-01-01,42183.913207,0.062500,7.80,4.0,150.0,0.0,1.0,43605.699637,1506.110026
2,abanca,2022-02-01,34498.153115,0.052288,7.25,4.0,150.0,0.0,2.0,48647.434439,32750.639361
3,abanca,2022-03-01,34546.319021,0.061503,6.75,4.0,150.0,0.0,2.0,26687.227413,15754.216964
4,abanca,2022-04-01,40508.288420,0.050222,6.75,4.0,300.0,0.0,2.0,4254.184995,6542.970638


In [680]:
products_data.columns

Index(['bank', 'date', 'web_traffic', 'cross_visitation', 'search_interest',
       'rank', 'incentive', 'apr', 'mentions', 'app_installs', 'ppc_spend'],
      dtype='object')

In [681]:
# cast date to datetime
products_data['date'] = pd.to_datetime(products_data['date'])

# log scale web traffic
products_data['web_traffic'] = np.log(products_data['web_traffic'])

# create a web_traffic_lag column
products_data['web_traffic_lag'] = products_data.groupby('bank')['web_traffic'].shift(1)

# deal with the first lag 0 value 
products_data['web_traffic_lag'] = products_data['web_traffic_lag'].fillna(0)

products_data = products_data[['bank', 'date', 'web_traffic_lag', 'web_traffic', 'cross_visitation', 'search_interest',
       'rank', 'mentions', 'app_installs', 'ppc_spend']]

data['date'] = pd.to_datetime(data['date']) 

In [682]:

# scale products data with standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

products_data[['web_traffic', 'web_traffic_lag', 'cross_visitation', 'search_interest', 'rank', 'mentions', 'app_installs', 'ppc_spend']] = scaler.fit_transform(products_data[['web_traffic', 'web_traffic_lag', 'cross_visitation', 'search_interest', 'rank', 'mentions', 'app_installs', 'ppc_spend']])

In [683]:

# attach product_data to data
data = pd.merge(products_data, data, on=['bank', 'date'], how='left')

# drop the 124 row 
data = data.drop(124)

data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))


# for some reason banks are not having the dates attaching properly to the bank data so we have to re add all of the dates to the banks

for bank in data.bank.unique():
    data.loc[data.bank == bank, 'date'] = list(products_data.date.unique())


In [684]:
# log scale web traffic
data['web_traffic'] = np.log(data['web_traffic'])


separate the data into increase apr and increase incentive lists 


In [685]:

data.tag = data.tag.fillna('')
apr_increase_dates = data[data.tag.str.contains('increase apr')]
incentive_increase_dates = data[data.tag.str.contains('increase incentive')]


all_dates = data.date.unique()


In [686]:
apr_increase_dates = apr_increase_dates[['date', 'bank', 'apr']]
incentive_increase_dates = incentive_increase_dates[['date', 'bank', 'incentive']]

In [687]:
banks = data.bank.unique()

### here we look for similar banks and average the 3 highest correlated banks that are not the bank of interest for the control bank

In [688]:
# Function to find top similar banks based on web_traffic similarity
def find_similar_banks(selected_bank, data, top_n=3):
    # Filter data for the selected bank and its web_traffic time series
    selected_series = data[data['bank'] == selected_bank]['web_traffic'].values
    
    # Calculate Pearson correlation coefficients with other banks
    similarities = []
    for bank in data['bank'].unique():
        if bank == selected_bank:
            continue
        series = data[data['bank'] == bank]['web_traffic'].values
        correlation = np.corrcoef(selected_series, series)[0, 1]  # Pearson correlation coefficient
        similarities.append((bank, correlation))
    
    # Sort banks by correlation coefficient in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Select top N banks (most similar)
    top_similar_banks = [similarity[0] for similarity in similarities[:top_n]]
    
    similar_bank_control = data[data.bank.isin(top_similar_banks)]

    return ' x '.join(top_similar_banks), pd.DataFrame(similar_bank_control.groupby('date')[['web_traffic','web_traffic_lag','cross_visitation', 'search_interest',
       'rank', 'mentions', 'app_installs', 'ppc_spend']].mean()).reset_index()
    

In [689]:
bank_control_dict = {}

for bank_name in data.bank.unique():
    bank_avgs, bank_df = find_similar_banks(bank_name, data, top_n=top_n_)
    bank_df['apr'] = 0
    bank_df['incentive'] = 0
    bank_control_dict[bank_name] = {'bank_avgs': bank_avgs, 'bank_df': bank_df}

In [690]:
# control_df_avg = data.loc[data.bank == 'ing']#pd.DataFrame(data.groupby('date')[['web_traffic','web_traffic_lag','cross_visitation', 'search_interest','rank', 'mentions', 'app_installs', 'ppc_spend']].mean()).reset_index()
# control_df_avg['apr'] = 0
# control_df_avg['incentive'] = 0

In [691]:
# reorder columns to match the bank_data columns
data = data[['date', 'web_traffic', 'web_traffic_lag', 'cross_visitation', 'search_interest', 'rank', 'mentions', 'app_installs', 'ppc_spend', 'apr', 'incentive', 'tag']]

here we create a dictionary for each bank as a key and each value is a dataframe of the bank's data.

In [692]:
# for each bank we take a subsection of the data that has no incentives 

bank_data_dict = {}

for bank in banks:
    # for each bank attach the relevant apr and incentive data
    
    apr_data = apr_increase_dates[apr_increase_dates.bank == bank].fillna(0)
    incentive_data = incentive_increase_dates[incentive_increase_dates.bank == bank].fillna(0)

    # merge the apr and incentives data with a copy of the products data specific to the bank
    temp_products_data = products_data[products_data.bank == bank].fillna(0)
    bank_data = temp_products_data.merge(apr_data, on=['date','bank'], how='left')
    bank_data = bank_data.merge(incentive_data, on=['date','bank'], how='left')

    bank_data_dict[bank] = bank_data


bank_data_dict
    

    

{'abanca':       bank       date  web_traffic_lag  web_traffic  cross_visitation  \
 0   abanca 2021-12-01        -4.435833    -0.048002         -0.427429   
 1   abanca 2022-01-01         0.166453     1.089653         -0.819808   
 2   abanca 2022-02-01         0.654223     0.865440         -1.078425   
 3   abanca 2022-03-01         0.558091     0.866995         -0.845045   
 4   abanca 2022-04-01         0.558758     1.044470         -1.130745   
 5   abanca 2022-05-01         0.634850     0.701919         -0.926270   
 6   abanca 2022-06-01         0.487982     0.884320         -1.013821   
 7   abanca 2022-07-01         0.566186     1.518390         -0.870214   
 8   abanca 2022-08-01         0.838043     1.363675         -0.613364   
 9   abanca 2022-09-01         0.771710     1.301022         -0.877671   
 10  abanca 2022-10-01         0.744847     1.291716         -0.336700   
 11  abanca 2022-11-01         0.740857     1.368661         -0.872920   
 12  abanca 2022-12-01      

filling na values


In [693]:
# drop bank_x and bank_y columns
for bank in banks:
    bank_data = bank_data_dict[bank]
    #bank_data = bank_data.drop(columns=['bank_x', 'bank_y'])
    bank_data.fillna(0, inplace=True)
    bank_data_dict[bank] = bank_data

here we create two functions to determine if the event is valid and another to find consecutive dates surrounding the event date 

In [694]:
def check_if_valid_event(bank_data, date, buffer, column):
    date_index = bank_data[bank_data.date == date].index[0]
    for i in range(1, buffer + 1):
        prev_date = bank_data.iloc[date_index - i].date
        next_date = bank_data.iloc[date_index + i].date
        prev_apr = bank_data.iloc[date_index - i][column]
        next_apr = bank_data.iloc[date_index + i][column]

        print(f"Checking date: {date}, Buffer-{i}: {prev_date} (APR: {prev_apr}), Buffer+{i}: {next_date} (APR: {next_apr})")
        
        if prev_apr != 0 or next_apr != 0:
            return False

    return True

def find_consecutive_dates(dates, buffer):
    # Calculate the required length of the sequence
    required_length = (buffer * 2) + 1
    
    # Convert dates to a sorted list of pandas Timestamps
    sorted_dates = sorted(pd.to_datetime(dates))
    
    # Iterate through the sorted dates list
    for i in range(len(sorted_dates) - required_length + 1):
        # Extract the sublist of the required length
        sublist = sorted_dates[i:i + required_length]
        
        
        # Check if the sublist is consecutive months
        if all((sublist[j + 1].year * 12 + sublist[j + 1].month) - (sublist[j].year * 12 + sublist[j].month) == 1 for j in range(len(sublist) - 1)):
            return sublist
    
    # If no such sequence is found
    return None

here we run a loop for each bank then find a control date in the 'ing' series which does not alter or change incentives or aprs.

this is for apr changes

In [695]:


# For each event date we verify that there is 1 day before and after that are not events
all_regressions_apr = {'regressions': []}

# For each bank we find an event date
for bank in banks:
    try:
        bank_data = bank_data_dict[bank]
        
        # Find dates where there is an APR increase
        apr_increase_dates = bank_data[bank_data.apr > 0]
        
        for date in apr_increase_dates.date:
            is_valid = check_if_valid_event(bank_data, date, buffer, 'apr')
            # If the date is valid we check for a control date in another bank
            if is_valid:
                bank_control = 'avg'
                bank_data_control = bank_control_dict[bank]['bank_df'][all_vars] # control_df_avg 
                control_date = bank_data_control[bank_data_control.date == date]
                if not control_date.empty and control_date.apr.iloc[0] == 0 and check_if_valid_event(bank_data_control, date, buffer, 'apr'):
                    # Now we have a valid event and control date
                    # We now need to find a control period where neither bank has activity

                    # Create a list of dates that are not events for either bank or the control period
                    viable_control_dates = bank_data_control[(bank_data_control.apr == 0) & (bank_data_control.incentive == 0)].date
                    # Also remove the event dates + buffer
                    viable_control_dates = viable_control_dates[viable_control_dates != date]
                    for i in range(1, buffer + 1):
                        viable_control_dates = viable_control_dates[viable_control_dates != date + pd.DateOffset(i)]
                        viable_control_dates = viable_control_dates[viable_control_dates != date - pd.DateOffset(i)]
                    
                    # Do the same for the bank dates
                    viable_bank_dates = bank_data[(bank_data.apr == 0) & (bank_data.incentive == 0)].date
                    viable_bank_dates = viable_bank_dates[viable_bank_dates != date]
                    for i in range(1, buffer + 1):
                        viable_bank_dates = viable_bank_dates[viable_bank_dates != date + pd.DateOffset(i)]
                        viable_bank_dates = viable_bank_dates[viable_bank_dates != date - pd.DateOffset(i)]

                    # Now we have a list of viable control dates for both banks
                    # We need to find the intersection of these two lists of dates
                    viable_periods = viable_control_dates[viable_control_dates.isin(viable_bank_dates)]

                    # Now we need to find a sequence of consecutive dates that are in the viable periods and (buffer*2)+1 dates long
                    control_period = find_consecutive_dates(viable_periods, buffer)
                    print(control_period)
                    
                    # Now we create each series with the relevant data
                    event_series = bank_data[(bank_data.date >= date - pd.DateOffset(months=buffer)) & (bank_data.date <= date + pd.DateOffset(months=buffer))]
                    event_control_series = bank_data_control[(bank_data_control.date >= date - pd.DateOffset(months=buffer)) & (bank_data_control.date <= date + pd.DateOffset(months=buffer))]

                    control_event_series = bank_data[(bank_data.date >= control_period[0]) & (bank_data.date <= control_period[-1])]
                    control_series = bank_data_control[(bank_data_control.date >= control_period[0]) & (bank_data_control.date <= control_period[-1])]

                    # Now we add it to a dictionary
                    regression = {
                        'banks_for_regression': f'{bank} * {"avg"}', 
                        'bank_of_interest_event_series': event_series, 
                        'control_bank_event_series': event_control_series, 
                        'bank_of_interest_non_event_series': control_event_series, 
                        'control_bank_non_event_series': control_series
                    }
                    all_regressions_apr['regressions'].append(regression)
    except Exception as e:
        print(f"Error processing bank {bank}: {e}")



Checking date: 2023-07-01 00:00:00, Buffer-1: 2023-06-01 00:00:00 (APR: 0.0), Buffer+1: 2023-08-01 00:00:00 (APR: 0.0)
Checking date: 2023-07-01 00:00:00, Buffer-2: 2023-05-01 00:00:00 (APR: 0.0), Buffer+2: 2023-09-01 00:00:00 (APR: 0.0)
Checking date: 2023-07-01 00:00:00, Buffer-1: 2023-06-01 00:00:00 (APR: 0), Buffer+1: 2023-08-01 00:00:00 (APR: 0)
Checking date: 2023-07-01 00:00:00, Buffer-2: 2023-05-01 00:00:00 (APR: 0), Buffer+2: 2023-09-01 00:00:00 (APR: 0)
[Timestamp('2022-05-01 00:00:00'), Timestamp('2022-06-01 00:00:00'), Timestamp('2022-07-01 00:00:00'), Timestamp('2022-08-01 00:00:00'), Timestamp('2022-09-01 00:00:00')]
Error processing bank abanca: single positional indexer is out-of-bounds
Checking date: 2023-07-01 00:00:00, Buffer-1: 2023-06-01 00:00:00 (APR: 0.0), Buffer+1: 2023-08-01 00:00:00 (APR: 0.0)
Checking date: 2023-07-01 00:00:00, Buffer-2: 2023-05-01 00:00:00 (APR: 0.0), Buffer+2: 2023-09-01 00:00:00 (APR: 0.0)
Checking date: 2023-07-01 00:00:00, Buffer-1: 2023

now we do the same for incentives 

In [696]:
products_data.columns

Index(['bank', 'date', 'web_traffic_lag', 'web_traffic', 'cross_visitation',
       'search_interest', 'rank', 'mentions', 'app_installs', 'ppc_spend'],
      dtype='object')

In [697]:
# For each event date we verify that there is 1 day before and after that are not events
all_regressions_incentives = {'regressions': []}

# For each bank we find an event date
for bank in banks:
    try:
        bank_data = bank_data_dict[bank]
        
        # Find dates where there is an APR increase
        incentive_increase_dates = bank_data[bank_data.incentive > 0]
        
        for date in incentive_increase_dates.date:
            is_valid = check_if_valid_event(bank_data, date, buffer, 'incentive')
            # If the date is valid we check for a control date in another bank
            if is_valid:
                bank_control = 'avg'
                # create a column that is the average of all banks that are not the bank of interest
                bank_data_control = bank_control_dict[bank]['bank_df'][all_vars] # control_df_avg
                print(bank_data_control)
                control_date = bank_data_control[bank_data_control.date == date]
                if not control_date.empty and control_date.incentive.iloc[0] == 0 and check_if_valid_event(bank_data_control, date, buffer, 'incentive'):
                    # Now we have a valid event and control date
                    # We now need to find a control period where neither bank has activity

                    # Create a list of dates that are not events for either bank or the control period
                    viable_control_dates = bank_data_control[(bank_data_control.incentive == 0) & (bank_data_control.incentive == 0)].date
                    # Also remove the event dates + buffer
                    viable_control_dates = viable_control_dates[viable_control_dates != date]
                    for i in range(1, buffer + 1):
                        viable_control_dates = viable_control_dates[viable_control_dates != date + pd.DateOffset(i)]
                        viable_control_dates = viable_control_dates[viable_control_dates != date - pd.DateOffset(i)]
                    
                    # Do the same for the bank dates
                    viable_bank_dates = bank_data[(bank_data.incentive == 0) & (bank_data.incentive == 0)].date
                    viable_bank_dates = viable_bank_dates[viable_bank_dates != date]
                    for i in range(1, buffer + 1):
                        viable_bank_dates = viable_bank_dates[viable_bank_dates != date + pd.DateOffset(i)]
                        viable_bank_dates = viable_bank_dates[viable_bank_dates != date - pd.DateOffset(i)]

                    # Now we have a list of viable control dates for both banks
                    # We need to find the intersection of these two lists of dates
                    viable_periods = viable_control_dates[viable_control_dates.isin(viable_bank_dates)]

                    # Now we need to find a sequence of consecutive dates that are in the viable periods and (buffer*2)+1 dates long
                    control_period = find_consecutive_dates(viable_periods, buffer)
                    print(control_period)
                    
                    # Now we create each series with the relevant data
                    event_series = bank_data[(bank_data.date >= date - pd.DateOffset(months=buffer)) & (bank_data.date <= date + pd.DateOffset(months=buffer))]
                    event_control_series = bank_data_control[(bank_data_control.date >= date - pd.DateOffset(months=buffer)) & (bank_data_control.date <= date + pd.DateOffset(months=buffer))]

                    control_event_series = bank_data[(bank_data.date >= control_period[0]) & (bank_data.date <= control_period[-1])]
                    control_series = bank_data_control[(bank_data_control.date >= control_period[0]) & (bank_data_control.date <= control_period[-1])]

                    # Now we add it to a dictionary
                    regression = {
                        'banks_for_regression': f'{bank} * {"avg"}', 
                        'bank_of_interest_event_series': event_series, 
                        'control_bank_event_series': event_control_series, 
                        'bank_of_interest_non_event_series': control_event_series, 
                        'control_bank_non_event_series': control_series
                    }
                    all_regressions_incentives['regressions'].append(regression)
    except Exception as e:
        print(f"Error processing bank {bank}: {e}")

all_regressions_incentives


Checking date: 2022-04-01 00:00:00, Buffer-1: 2022-03-01 00:00:00 (APR: 0.0), Buffer+1: 2022-05-01 00:00:00 (APR: 0.0)
Checking date: 2022-04-01 00:00:00, Buffer-2: 2022-02-01 00:00:00 (APR: 0.0), Buffer+2: 2022-06-01 00:00:00 (APR: 0.0)
    web_traffic_lag  cross_visitation  search_interest  web_traffic  \
0         -4.435833          0.259504         0.010759    -0.384480   
1          0.105725          0.169191         0.006819    -0.533463   
2          0.073072         -0.096774        -0.002916    -0.419021   
3          0.086699         -0.018262        -0.066660    -0.395130   
4          0.073713         -0.085661         0.356597    -0.420710   
5          0.058999         -0.092540         0.497992    -0.022895   
6          0.058343         -0.085726         0.436103    -0.337995   
7          0.076620          0.262062         0.443520    -0.310457   
8          0.174722          0.232186         0.299344    -0.065186   
9          0.254133          0.016658         0.2729

{'regressions': [{'banks_for_regression': 'abanca * avg',
   'bank_of_interest_event_series':      bank       date  web_traffic_lag  web_traffic  cross_visitation  \
   2  abanca 2022-02-01         0.654223     0.865440         -1.078425   
   3  abanca 2022-03-01         0.558091     0.866995         -0.845045   
   4  abanca 2022-04-01         0.558758     1.044470         -1.130745   
   5  abanca 2022-05-01         0.634850     0.701919         -0.926270   
   6  abanca 2022-06-01         0.487982     0.884320         -1.013821   
   
      search_interest      rank  mentions  app_installs  ppc_spend  apr  \
   2        -0.710122 -0.380274 -0.071236      0.224657   0.256398  0.0   
   3        -0.733302 -0.380274 -0.071236     -0.365729  -0.475698  0.0   
   4        -0.733302 -0.380274 -0.071236     -0.968828  -0.872459  0.0   
   5        -0.703168 -0.600442 -0.071236     -0.064996   0.403430  0.0   
   6        -0.652174 -0.380274 -0.922968      2.374108   0.043365  0.0   
   
 

define a function to get the relevant data from the statsmodel output into a dataframe

In [698]:
def extract_data_from_sm(model):
    model_dict = {}

    # Extracting coefficients
    coefficients = model.params
    model_dict['coefficients'] = coefficients

    # Extracting standard errors of coefficients
    std_errors = model.bse
    model_dict['standard_errors'] = std_errors

    # Extracting t-values
    t_values = model.tvalues
    model_dict['t_values'] = t_values

    # Extracting p-values
    p_values = model.pvalues
    model_dict['p_values'] = p_values

    # Extracting R-squared
    r_squared = model.rsquared
    model_dict['r_squared'] = r_squared

    return pd.DataFrame(model_dict)

define a function to get control and event periods for each bank out of the dictionary and into a dataframe that be regressed 

also creates the interation terms for the regression

In [699]:
def get_regression_summary(regression): 
    # Extract the DataFrames
    bank_of_interest_event_series = regression['bank_of_interest_event_series']
    control_bank_event_series = regression['control_bank_event_series']
    bank_of_interest_non_event_series = regression['bank_of_interest_non_event_series']
    control_bank_non_event_series = regression['control_bank_non_event_series']

    # standardize columns to [['date', 'web_traffic', 'web_traffic_lag', 'cross_visitation', 'search_interest', 'rank', 'mentions', 'app_installs', 'ppc_spend', 'apr', 'incentive', 'tag']]
    bank_of_interest_event_series = bank_of_interest_event_series[all_vars]
    control_bank_event_series = control_bank_event_series[all_vars]
    bank_of_interest_non_event_series = bank_of_interest_non_event_series[all_vars]
    control_bank_non_event_series = control_bank_non_event_series[all_vars]
    
    # remove 3rd column
    control_bank_event_series = control_bank_event_series.loc[:,~control_bank_event_series.columns.duplicated()].copy()
    control_bank_non_event_series = control_bank_non_event_series.loc[:,~control_bank_non_event_series.columns.duplicated()].copy()

    # print all df shapes 
    print(bank_of_interest_event_series.index)
    print(control_bank_event_series.index)
    print(bank_of_interest_non_event_series.index)
    print(control_bank_non_event_series.index)

    # Add a 'type' and 'period' columns to each DataFrame
    bank_of_interest_event_series['type'] = 'bank_of_interest_event'
    control_bank_event_series['type'] = 'control_bank_event'
    bank_of_interest_non_event_series['type'] = 'bank_of_interest_non_event'
    control_bank_non_event_series['type'] = 'control_bank_non_event'

    bank_of_interest_event_series['period'] = 'post'
    control_bank_event_series['period'] = 'post'
    bank_of_interest_non_event_series['period'] = 'pre'
    control_bank_non_event_series['period'] = 'pre'

    # Combine the series into a single DataFrame
    combined_df = pd.concat([
        bank_of_interest_event_series,
        control_bank_event_series,
        bank_of_interest_non_event_series,
        control_bank_non_event_series
    ])

    # Reset index to avoid duplicate indices
    combined_df.reset_index(drop=True, inplace=True)
    

    # Create dummy variables
    combined_df['treatment'] = combined_df['type'].apply(lambda x: 1 if x in ['bank_of_interest_event', 'bank_of_interest_non_event'] else 0)
    combined_df['post'] = combined_df['period'].apply(lambda x: 1 if x == 'post' else 0)


    # Interaction term: treatment * post
    combined_df['treatment_post'] = combined_df['treatment'] * combined_df['post']

    # Dependent variable
    y = combined_df['web_traffic']

    # Independent variables
    X = combined_df[ind_vars + ['treatment', 'post', 'treatment_post']]
    X = sm.add_constant(X)

    # Fit the model
    model = sm.OLS(y, X).fit()

    # Get the summary of the regression
    summary = extract_data_from_sm(model)
    summary['output_title'] = regression['banks_for_regression']
    summary['date_of_event'] = bank_of_interest_event_series.date.median()

    # Parallel Trends Check: Only pre-treatment periods
    pre_treatment_df = combined_df[combined_df['period'] == 'pre']
    y_pre = pre_treatment_df['web_traffic']
    X_pre = pre_treatment_df[['treatment', 'post']]  # Only use 'treatment' as 'post' is always 0
    X_pre = sm.add_constant(X_pre)
    parallel_model = sm.OLS(y_pre, X_pre).fit()
    parallel_summary = extract_data_from_sm(parallel_model)
    parallel_summary['regression_type'] = 'Parallel Trends Check'
    parallel_summary['output_title'] = regression['banks_for_regression']

    return summary, parallel_summary

### get regrssion outputs for each regression for incentives

In [700]:
all_regression_outputs = []
all_parallel_trends_incentives = []

for regression in all_regressions_incentives['regressions']:
    if len(regression.keys()) > 0:
        summary, parallel_trends = get_regression_summary(regression)
        all_regression_outputs.append(summary)
        all_parallel_trends_incentives.append(parallel_trends)
    else: 
        print('No valid regressions found')
        print(regression)


Index([2, 3, 4, 5, 6], dtype='int64')
Index([2, 3, 4, 5, 6], dtype='int64')
Index([5, 6, 7, 8, 9], dtype='int64')
Index([5, 6, 7, 8, 9], dtype='int64')
Index([6, 7, 8, 9, 10], dtype='int64')
Index([6, 7, 8, 9, 10], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([14, 15, 16, 17, 18], dtype='int64')
Index([14, 15, 16, 17, 18], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([3, 4, 5, 6, 7], dtype='int64')
Index([3, 4, 5, 6, 7], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([6, 7, 8, 9, 10], dtype='int64')
Index([6, 7, 8, 9, 10], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([19, 20, 21, 22, 23], dtype='int64')
Index([19, 20, 21, 22, 23], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')


visualize the regression outputs for each bank

In [701]:
all_incentive_diff_n_diff = pd.concat(all_regression_outputs)

# create a column from the index 
all_incentive_diff_n_diff['variable_name'] = all_incentive_diff_n_diff.index

all_incentive_diff_n_diff = all_incentive_diff_n_diff.reset_index(drop=True)

all_incentive_diff_n_diff['normalized_coefficients'] = np.exp(all_incentive_diff_n_diff['coefficients'])

all_incentive_diff_n_diff

,coefficients,standard_errors,t_values,p_values,r_squared,output_title,date_of_event,variable_name,normalized_coefficients
0,-0.401846,0.225282,-1.783750,0.097812,0.924686,abanca * avg,2022-04-01,const,0.669084
1,0.967011,0.713168,1.355936,0.198206,0.924686,abanca * avg,2022-04-01,web_traffic_lag,2.630071
2,-0.016154,0.493129,-0.032758,0.974365,0.924686,abanca * avg,2022-04-01,cross_visitation,0.983976
3,0.386163,0.418364,0.923031,0.372806,0.924686,abanca * avg,2022-04-01,search_interest,1.471325
4,1.167844,0.822404,1.420037,0.179132,0.924686,abanca * avg,2022-04-01,treatment,3.215053
5,-0.080689,0.172829,-0.466871,0.648320,0.924686,abanca * avg,2022-04-01,post,0.922480
6,-0.115712,0.207654,-0.557235,0.586834,0.924686,abanca * avg,2022-04-01,treatment_post,0.890732
7,-0.000320,0.113821,-0.002814,0.997797,0.753579,banc sabadell * avg,2022-08-01,const,0.999680
8,0.000982,0.037451,0.026223,0.979478,0.753579,banc sabadell * avg,2022-08-01,web_traffic_lag,1.000983
9,0.282505,0.303296,0.931450,0.368597,0.753579,banc sabadell * avg,2022-08-01,cross_visitation,1.326448


get average statistics

In [702]:

print(all_incentive_diff_n_diff.coefficients.mean())
print(all_incentive_diff_n_diff.p_values.mean())


0.1618226615592423
0.4138568987441562


In [703]:
all_incentive_diff_n_diff.loc[all_incentive_diff_n_diff['variable_name'] == 'treatment_post', 'coefficients'].mean()

0.49603258128165256

view pertinent statistics

In [704]:

all_incentive_diff_n_diff.loc[all_incentive_diff_n_diff.variable_name == 'treatment_post', ['normalized_coefficients', 'coefficients','p_values', 'variable_name','output_title', 'date_of_event', 'r_squared']].to_csv('data/incentive_diff_n_diff.csv')
all_incentive_diff_n_diff.loc[all_incentive_diff_n_diff.variable_name == 'treatment_post', ['normalized_coefficients', 'coefficients','p_values', 'variable_name', 'output_title', 'date_of_event', 'r_squared']]


,normalized_coefficients,coefficients,p_values,variable_name,output_title,date_of_event,r_squared
6,0.890732,-0.115712,0.586834,treatment_post,abanca * avg,2022-04-01,0.924686
13,1.532968,0.427206,0.055874,treatment_post,banc sabadell * avg,2022-08-01,0.753579
20,3.896109,1.359978,0.000001,treatment_post,banc sabadell * avg,2023-04-01,0.965658
27,1.252974,0.225520,0.045527,treatment_post,santander * avg,2022-05-01,0.966405
34,1.468991,0.384576,0.042643,treatment_post,santander * avg,2022-08-01,0.934748
41,2.002963,0.694628,0.155299,treatment_post,santander * avg,2023-09-01,0.794669


In [705]:
all_parallel_trends_incentives = pd.concat(all_parallel_trends_incentives)
all_parallel_trends_incentives

,coefficients,standard_errors,t_values,p_values,r_squared,regression_type,output_title
const,-0.131872,0.123477,-1.067990,3.166894e-01,0.871409,Parallel Trends Check,abanca * avg
treatment,1.285738,0.174623,7.362934,7.895133e-05,0.871409,Parallel Trends Check,abanca * avg
post,0.000000,0.000000,NaN,NaN,0.871409,Parallel Trends Check,abanca * avg
const,0.016330,0.018576,0.879123,4.049659e-01,0.914043,Parallel Trends Check,banc sabadell * avg
treatment,0.242300,0.026270,9.223352,1.546970e-05,0.914043,Parallel Trends Check,banc sabadell * avg
post,0.000000,0.000000,NaN,NaN,0.914043,Parallel Trends Check,banc sabadell * avg
const,0.016330,0.018576,0.879123,4.049659e-01,0.914043,Parallel Trends Check,banc sabadell * avg
treatment,0.242300,0.026270,9.223352,1.546970e-05,0.914043,Parallel Trends Check,banc sabadell * avg
post,0.000000,0.000000,NaN,NaN,0.914043,Parallel Trends Check,banc sabadell * avg
const,-0.350038,0.032479,-10.777405,4.840714e-06,0.976862,Parallel Trends Check,santander * avg


### get regrssion outputs for each regression for aprs

In [706]:
all_regression_outputs_apr = []
all_parallel_trends_apr = []

for regression in all_regressions_apr['regressions']:
    if len(regression.keys()) > 0:
        summary, parallel_trends = get_regression_summary(regression)
        all_regression_outputs_apr.append(summary)
        all_parallel_trends_apr.append(parallel_trends)
    else: 
        print('No valid regressions found')
        print(regression)


Index([17, 18, 19, 20, 21], dtype='int64')
Index([17, 18, 19, 20, 21], dtype='int64')
Index([5, 6, 7, 8, 9], dtype='int64')
Index([5, 6, 7, 8, 9], dtype='int64')
Index([17, 18, 19, 20, 21], dtype='int64')
Index([17, 18, 19, 20, 21], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([6, 7, 8, 9, 10], dtype='int64')
Index([6, 7, 8, 9, 10], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([15, 16, 17, 18, 19], dtype='int64')
Index([15, 16, 17, 18, 19], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([0, 1, 2, 3, 4], dtype='int64')
Index([2, 3, 4, 5, 6], dtype='int64')
Index([2, 3, 4, 5, 6], dtype='int64')
Index([5, 6, 7, 8, 9], dtype='int64')
Index([5, 6, 7, 8, 9], dtype='int64')
Index([14, 15, 16, 17, 18], dtype='int64')
Index([14, 15, 16, 17, 18], dtype='int64')
Index([5, 6, 7, 8, 9], dtype='int64')
Index([5, 6, 7, 8, 9], dtype='int64')
Index([20, 21, 22, 23, 24], dtype='int64')
Ind

view results as a dataframe

In [707]:
all_incentive_diff_n_diff_2 = pd.concat(all_regression_outputs_apr)

# create a column from the index 
all_incentive_diff_n_diff_2['variable_name'] = all_incentive_diff_n_diff_2.index

all_incentive_diff_n_diff_2 = all_incentive_diff_n_diff_2.reset_index(drop=True)

all_incentive_diff_n_diff_2['normalized_coefficients'] = np.exp(all_incentive_diff_n_diff_2['coefficients'])

all_incentive_diff_n_diff_2

,coefficients,standard_errors,t_values,p_values,r_squared,output_title,date_of_event,variable_name,normalized_coefficients
0,-0.068418,0.390611,-0.175156,0.863656,0.910404,abanca * avg,2023-07-01,const,0.933870
1,-0.099217,1.039183,-0.095476,0.925393,0.910404,abanca * avg,2023-07-01,web_traffic_lag,0.905546
2,0.639639,0.630182,1.015007,0.328622,0.910404,abanca * avg,2023-07-01,cross_visitation,1.895796
3,-0.240142,0.808362,-0.297073,0.771105,0.910404,abanca * avg,2023-07-01,search_interest,0.786516
4,1.673818,1.396370,1.198692,0.252043,0.910404,abanca * avg,2023-07-01,treatment,5.332489
...,...,...,...,...,...,...,...,...,...
65,0.605720,0.402032,1.506648,0.155810,0.672176,evobanco * avg,2023-04-01,cross_visitation,1.832572
66,0.293257,0.810407,0.361864,0.723268,0.672176,evobanco * avg,2023-04-01,search_interest,1.340788
67,-0.026100,0.705805,-0.036979,0.971063,0.672176,evobanco * avg,2023-04-01,treatment,0.974238
68,-0.145272,0.232066,-0.625994,0.542154,0.672176,evobanco * avg,2023-04-01,post,0.864787


view average results 

In [708]:
print(all_incentive_diff_n_diff_2.coefficients.mean())
print(all_incentive_diff_n_diff_2.p_values.mean())



0.06302981918416539
0.4100528667630663


In [709]:
all_parallel_trends_apr = pd.concat(all_parallel_trends_apr)

view important resuls

In [710]:
all_parallel_trends_apr.loc[all_parallel_trends_apr.p_values > 0.05]

,coefficients,standard_errors,t_values,p_values,r_squared,regression_type,output_title
const,-0.131872,0.123477,-1.067990,0.316689,0.871409,Parallel Trends Check,abanca * avg
const,0.016330,0.018576,0.879123,0.404966,0.914043,Parallel Trends Check,banc sabadell * avg
const,0.016330,0.018576,0.879123,0.404966,0.914043,Parallel Trends Check,banc sabadell * avg
const,-0.078716,0.077106,-1.020883,0.337186,0.401778,Parallel Trends Check,openbank * avg
const,-0.078716,0.077106,-1.020883,0.337186,0.401778,Parallel Trends Check,openbank * avg
const,-0.078716,0.077106,-1.020883,0.337186,0.401778,Parallel Trends Check,openbank * avg


In [711]:
(99+99+95)/3

97.66666666666667

In [712]:

all_incentive_diff_n_diff_2.loc[all_incentive_diff_n_diff_2.variable_name == 'treatment_post', ['coefficients','p_values', 'output_title', 'date_of_event', 'r_squared']].to_csv('data/apr_diff_n_diff.csv')
apr_increase_coeffs = all_incentive_diff_n_diff_2.loc[all_incentive_diff_n_diff_2.variable_name == 'treatment_post', ['coefficients','p_values', 'output_title', 'date_of_event', 'r_squared']]
apr_increase_coeffs


,coefficients,p_values,output_title,date_of_event,r_squared
6,1.382451,0.007266,abanca * avg,2023-07-01,0.910404
13,0.333641,0.248464,n26 * avg,2023-07-01,0.432644
20,0.427206,0.055874,banc sabadell * avg,2022-08-01,0.753579
27,0.959336,0.002334,banc sabadell * avg,2023-05-01,0.846371
34,0.129655,0.417713,openbank * avg,2022-04-01,0.669769
41,-0.088900,0.744717,openbank * avg,2023-04-01,0.575247
48,-0.192545,0.735670,openbank * avg,2023-10-01,0.430702
55,0.085877,0.764392,myinvestor * avg,2023-01-01,0.823845
62,-0.420335,0.051959,evobanco * avg,2022-11-01,0.925183
69,0.445199,0.151340,evobanco * avg,2023-04-01,0.672176


### modify function to take average of all banks not included in the time period specified rather than just ing


In [713]:
incentive_increase_coeffs = all_incentive_diff_n_diff.loc[all_incentive_diff_n_diff.variable_name == 'treatment_post', ['normalized_coefficients', 'coefficients','p_values', 'output_title', 'date_of_event', 'r_squared']]
print(incentive_increase_coeffs.normalized_coefficients.mean())
print(incentive_increase_coeffs.p_values.mean())

1.8407894862634133
0.1476961657693354


In [714]:
all_parallel_trends_incentives.loc[all_parallel_trends_incentives.p_values > 0.05]


,coefficients,standard_errors,t_values,p_values,r_squared,regression_type,output_title
const,-0.131872,0.123477,-1.067990,0.316689,0.871409,Parallel Trends Check,abanca * avg
const,0.016330,0.018576,0.879123,0.404966,0.914043,Parallel Trends Check,banc sabadell * avg
const,0.016330,0.018576,0.879123,0.404966,0.914043,Parallel Trends Check,banc sabadell * avg


In [719]:
incentive_increase_coeffs

,normalized_coefficients,coefficients,p_values,output_title,date_of_event,r_squared
6,0.890732,-0.115712,0.586834,abanca * avg,2022-04-01,0.924686
13,1.532968,0.427206,0.055874,banc sabadell * avg,2022-08-01,0.753579
20,3.896109,1.359978,0.000001,banc sabadell * avg,2023-04-01,0.965658
27,1.252974,0.225520,0.045527,santander * avg,2022-05-01,0.966405
34,1.468991,0.384576,0.042643,santander * avg,2022-08-01,0.934748
41,2.002963,0.694628,0.155299,santander * avg,2023-09-01,0.794669


In [716]:
apr_increases_coeffs = all_incentive_diff_n_diff_2.loc[all_incentive_diff_n_diff_2.variable_name == 'treatment_post', ['normalized_coefficients', 'coefficients','p_values', 'output_title', 'date_of_event', 'r_squared']]
print(apr_increases_coeffs.normalized_coefficients.mean())
print(apr_increases_coeffs.p_values.mean())

1.5709159722649642
0.31797309195210965


In [717]:
# drop normalized coefficients
apr_increase_coeffs

,coefficients,p_values,output_title,date_of_event,r_squared
6,1.382451,0.007266,abanca * avg,2023-07-01,0.910404
13,0.333641,0.248464,n26 * avg,2023-07-01,0.432644
20,0.427206,0.055874,banc sabadell * avg,2022-08-01,0.753579
27,0.959336,0.002334,banc sabadell * avg,2023-05-01,0.846371
34,0.129655,0.417713,openbank * avg,2022-04-01,0.669769
41,-0.088900,0.744717,openbank * avg,2023-04-01,0.575247
48,-0.192545,0.735670,openbank * avg,2023-10-01,0.430702
55,0.085877,0.764392,myinvestor * avg,2023-01-01,0.823845
62,-0.420335,0.051959,evobanco * avg,2022-11-01,0.925183
69,0.445199,0.151340,evobanco * avg,2023-04-01,0.672176


In [718]:
# find only with significant p values
all_parallel_trends_apr.loc[all_parallel_trends_apr.p_values > 0.05]

,coefficients,standard_errors,t_values,p_values,r_squared,regression_type,output_title
const,-0.131872,0.123477,-1.067990,0.316689,0.871409,Parallel Trends Check,abanca * avg
const,0.016330,0.018576,0.879123,0.404966,0.914043,Parallel Trends Check,banc sabadell * avg
const,0.016330,0.018576,0.879123,0.404966,0.914043,Parallel Trends Check,banc sabadell * avg
const,-0.078716,0.077106,-1.020883,0.337186,0.401778,Parallel Trends Check,openbank * avg
const,-0.078716,0.077106,-1.020883,0.337186,0.401778,Parallel Trends Check,openbank * avg
const,-0.078716,0.077106,-1.020883,0.337186,0.401778,Parallel Trends Check,openbank * avg


In [491]:
ind_vars = ['web_traffic_lag',  'cross_visitation', 'search_interest']
all_vars = ind_vars + ['web_traffic', 'date', 'incentive', 'apr']

top_n_ = 5
buffer = 2

In [650]:
# supress warnings 
import warnings

warnings.filterwarnings('ignore')

In [665]:
for test_regression in all_regressions_apr['regressions']:
    print(test_regression['banks_for_regression'])
    bank_of_interest_event_series = test_regression['bank_of_interest_event_series']    
    control_bank_event_series = test_regression['control_bank_event_series']
    bank_of_interest_non_event_series = test_regression['bank_of_interest_non_event_series']
    control_bank_non_event_series = test_regression['control_bank_non_event_series']

    bank_of_interest_event_series['series'] = 'Bank of Interest Event'
    control_bank_event_series['series'] = 'Control Bank Event'
    bank_of_interest_non_event_series['series'] = 'Bank of Interest Non Event'
    control_bank_non_event_series['series'] = 'Control Bank Non Event'

     # Add a 'type' and 'period' columns to each DataFrame
    bank_of_interest_event_series['type'] = 'bank_of_interest_event'
    control_bank_event_series['type'] = 'control_bank_event'
    bank_of_interest_non_event_series['type'] = 'bank_of_interest_non_event'
    control_bank_non_event_series['type'] = 'control_bank_non_event'

    bank_of_interest_event_series['period'] = 'post'
    control_bank_event_series['period'] = 'post'
    bank_of_interest_non_event_series['period'] = 'pre'
    control_bank_non_event_series['period'] = 'pre'

    # Concatenate all dataframes into one
    combined_df = pd.concat([
        bank_of_interest_event_series,
        control_bank_event_series,
        bank_of_interest_non_event_series,
        control_bank_non_event_series
    ])

    

    # Plot the combined dataframe
    fig = px.line(combined_df, x='date', y='web_traffic', color='type', title='Web Traffic Across Different Series - Incentive')

    # change legend
    fig.update_layout(legend_title_text='Series')
    


    fig.show()


abanca * avg


n26 * avg


banc sabadell * avg


banc sabadell * avg


openbank * avg


openbank * avg


openbank * avg


myinvestor * avg


evobanco * avg


evobanco * avg


In [666]:
for test_regression in all_regressions_apr['regressions']:
    print(test_regression['banks_for_regression'])
    bank_of_interest_event_series = test_regression['bank_of_interest_event_series']    
    control_bank_event_series = test_regression['control_bank_event_series']
    bank_of_interest_non_event_series = test_regression['bank_of_interest_non_event_series']
    control_bank_non_event_series = test_regression['control_bank_non_event_series']

    bank_of_interest_event_series['series'] = 'Bank of Interest Event'
    control_bank_event_series['series'] = 'Control Bank Event'
    bank_of_interest_non_event_series['series'] = 'Bank of Interest Non Event'
    control_bank_non_event_series['series'] = 'Control Bank Non Event'

     # Add a 'type' and 'period' columns to each DataFrame
    bank_of_interest_event_series['type'] = 'bank_of_interest_event'
    control_bank_event_series['type'] = 'control_bank_event'
    bank_of_interest_non_event_series['type'] = 'bank_of_interest_non_event'
    control_bank_non_event_series['type'] = 'control_bank_non_event'

    bank_of_interest_event_series['period'] = 'post'
    control_bank_event_series['period'] = 'post'
    bank_of_interest_non_event_series['period'] = 'pre'
    control_bank_non_event_series['period'] = 'pre'

    # Concatenate all dataframes into one
    combined_df = pd.concat([
        bank_of_interest_event_series,
        control_bank_event_series,
        bank_of_interest_non_event_series,
        control_bank_non_event_series
    ])

    

    # Plot the combined dataframe
    fig = px.line(combined_df, x='date', y='web_traffic', color='type', title='Web Traffic Across Different Series - APR')

    # change legend
    fig.update_layout(legend_title_text='Series')
    


    fig.show()


abanca * avg


n26 * avg


banc sabadell * avg


banc sabadell * avg


openbank * avg


openbank * avg


openbank * avg


myinvestor * avg


evobanco * avg


evobanco * avg
